In [ ]:
import os
import cv2
import glob
import shutil

import pandas as pd
import pathlib
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from PIL import Image as ImagePIL

from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential

# Custom Funcs
from Unpack_Scaffold_Data import readAndOutputDataset, curveVisualization

# Data Read Utility

In [ ]:
curve_path = "/Users/zacharyg/Documents/GitHub/fundemental-neural-nets/GANS/Scaffold_GAN/scaffold_dataset_WU_LAB/Prints"
modulus_path = "/Users/zacharyg/Documents/GitHub/fundemental-neural-nets/GANS/Scaffold_GAN/scaffold_dataset_WU_LAB/Prints/modulus_data_types.csv"


In [ ]:
X, y, y_df, file_order = readAndOutputDataset(curve_path, modulus_path, reverse=True);

In [ ]:
# Sanity Check
print("X SHAPE:", X.shape);
print("y SHAPE:", y.shape);
print();


# Visualization
curveVisualization(X, y, file_order);

# Utility

In [ ]:
def transposeStressData(X_Data):
    X = [];
    
    for data in X_Data:
        X.append(data.T);
        
    return np.array(X);

def normalizeStressStrain(x):
    for curve_index in range(len(x)):
        curve = x[curve_index];
        
        max_stress_val = np.max(curve[0]);
        max_strain_val = np.max(curve[1]);
        
        curve[0] = curve[0] / max_stress_val;
        curve[1] = curve[1] / max_strain_val;
        
    return x;

def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    
    Parameters
    -----------------
    x: Array of Homogenous (RGB) values of input data 
    
    Returns
    -----------------
    new_imgs: (numpy integer array) Numpy array of normalized data
    """
    return np.array((x - np.min(x)) / (np.max(x) - np.min(x)))

def stringtoCategorical(y):    
    data = [];
    
    for type_index in range(len(y)):
        wrd = y[type_index];
        encoding = 0.0;
        
        if (wrd == "Cubic"):
            encoding = 1.0;
        elif (wrd == "Gyroid"):
            encoding = 2.0;
            
        data.append([encoding]);
        
    return np.array(data);

# 3D Processing Parameter Stripping

In [ ]:
def parameterStrip(y):
    y_t = y.T;
    
    Index = y_t[0];
    Modulus = y_t[1];
    Spacing = y_t[2];
    Infill = y_t[3];
    Height = y_t[4];
    Speed = y_t[5];
    Temp = y_t[6];
    Mass = y_t[7];
    Porosity = y_t[8];
    Type = y_t[9];
    return Index, Modulus, Spacing, Infill, Height, Speed, Temp, Mass, Porosity, Type

Index, Modulus, Spacing, Infill, Height, Speed, Temp, Mass, Porosity, Type = parameterStrip(y);


# Stress-Strain preprocessing

In [ ]:
# Normalize the X data
X_norm = normalizeStressStrain(X);


# LSTM -> Dense (Sequential Classification)

In [ ]:
X_norm_T = transposeStressData(X_norm); # This is for LSTM, needs to be (sample, timesteps, features);
print(X_norm_T.shape);

# To categorical
y = stringtoCategorical(Type);

## Split

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X_norm_T, y, test_size=0.33, random_state=42);
X_train, X_test, y_train, y_test = train_test_split(X_norm_T, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

# One hot encoding
y_train = tf.keras.utils.to_categorical(y_train);
y_test = tf.keras.utils.to_categorical(y_test);
y_val = tf.keras.utils.to_categorical(y_val);

print(y_train);

# Sanity Check:
print("X_Train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print();
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
print();
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

## LSTM Model

In [ ]:
def LSTM():
    model = Sequential();
    model.add(tf.keras.layers.LSTM(2, input_shape=(1803, 2), return_sequences=True));
    model.add(tf.keras.layers.LSTM(2, input_shape=(1803, 2), return_sequences=False));
    model.add(tf.keras.layers.Dense(3, activation="softmax"));
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(), 
        optimizer=opt, 
        metrics=['accuracy'],
    );
    
    return model;



# Training

In [ ]:
LSTM_Model = LSTM();
LSTM_Model.summary();


In [ ]:
history = LSTM_Model.fit(
    X_train, 
    y_train, 
    epochs=2, 
    batch_size=1,
    validation_data=(X_val, y_val)
);

# Graphs

In [ ]:
print(history.history.keys())

#  "Accuracy"
plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# "Loss"
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Save Model

In [ ]:
# MAKE SURE TO MODIFY MODEL NAME AND FOLLOW THE NAMING FORMAT!
# Naming Format: feature_LSTM

model_name = "Type_LSTM"
save_path = "/Users/zacharyg/Documents/GitHub/fundemental-neural-nets/GANS/Scaffold_GAN/" + model_name
if not os.path.exists(save_path):
    os.makedirs(save_path);
    
LSTM_Model.save(save_path);